In [3]:
import pandas as pd 


df = pd.read_csv("../output/sim_history.csv")
df

,month,oil_price,demand_ns,demand_gom,company,cash_musd,rigs_active,rigs_warm,rigs_cold
0,1,68.72,1.16,1.70,PlayerCo,56.11,1,1,0
1,1,68.72,1.16,1.70,Stack&Pray Drilling,44.71,2,0,0
2,1,68.72,1.16,1.70,Bluewater Titans,84.37,0,1,1
3,2,71.38,1.24,1.82,PlayerCo,60.07,2,0,0
4,2,71.38,1.24,1.82,Stack&Pray Drilling,49.42,2,0,0
...,...,...,...,...,...,...,...,...,...
67,23,59.47,1.40,2.06,Stack&Pray Drilling,158.23,2,0,0
68,23,59.47,1.40,2.06,Bluewater Titans,107.47,1,1,0
69,24,72.43,1.41,2.07,PlayerCo,153.88,2,0,0
70,24,72.43,1.41,2.07,Stack&Pray Drilling,161.95,1,1,0


In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Ensure types
df["month"] = df["month"].astype(int)
for c in ["oil_price", "demand_ns", "demand_gom", "cash_musd"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Derived metrics
df["rigs_total"] = df["rigs_active"] + df["rigs_warm"] + df["rigs_cold"]
df["utilisation"] = np.where(df["rigs_total"] > 0, df["rigs_active"] / df["rigs_total"], np.nan)

# Market table (unique per month)
market = (
    df[["month", "oil_price", "demand_ns", "demand_gom"]]
    .drop_duplicates()
    .sort_values("month")
)

# Latest snapshot per company
latest = df.sort_values("month").groupby("company").tail(1).sort_values("cash_musd", ascending=False)

# ================
# CHARTS
# ================

# 1) Oil price over time
fig = px.line(market, x="month", y="oil_price", title="Oil price over time")
fig.show()

# 2) Demand over time (both regions)
m2 = market.melt(id_vars=["month"], value_vars=["demand_ns", "demand_gom"], var_name="region", value_name="demand")
fig = px.line(m2, x="month", y="demand", color="region", title="Regional demand over time")
fig.show()

# 3) Company cash over time
fig = px.line(df, x="month", y="cash_musd", color="company", title="Cash over time (by company)")
fig.show()

# 4) Cash indexed to 100 at first month (growth view)
base = df.sort_values("month").groupby("company").first().reset_index()[["company", "cash_musd"]].rename(columns={"cash_musd": "cash0"})
df_idx = df.merge(base, on="company", how="left")
df_idx["cash_index"] = 100 * df_idx["cash_musd"] / df_idx["cash0"]
fig = px.line(df_idx, x="month", y="cash_index", color="company", title="Cash indexed (start = 100)")
fig.show()

# 5) Utilisation over time
fig = px.line(df, x="month", y="utilisation", color="company", title="Utilisation over time (active / total rigs)")
fig.update_yaxes(tickformat=".0%")
fig.show()

# 6) Fleet state counts over time (stacked area per company)
long_rigs = df.melt(
    id_vars=["month", "company"],
    value_vars=["rigs_active", "rigs_warm", "rigs_cold"],
    var_name="state",
    value_name="count",
)
fig = px.area(
    long_rigs,
    x="month",
    y="count",
    color="state",
    facet_row="company",
    title="Fleet state over time (stacked, per company)",
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

# 7) End-of-sim cash bar chart
fig = px.bar(latest, x="company", y="cash_musd", title="End-of-sim cash ($m) by company")
fig.show()

# 8) Scatter: oil price vs demand (lag intuition)
fig = px.scatter(market, x="oil_price", y="demand_ns", title="Oil price vs Demand (North Sea)", trendline="ols")
fig.show()
fig = px.scatter(market, x="oil_price", y="demand_gom", title="Oil price vs Demand (GOM)", trendline="ols")
fig.show()

# 9) Scatter: demand vs cash (per company)
fig = px.scatter(
    df,
    x="demand_gom",
    y="cash_musd",
    color="company",
    title="Cash vs Demand (GOM) by company",
)
fig.show()

# 10) Heatmap: cash by company & month
pivot_cash = df.pivot_table(index="company", columns="month", values="cash_musd", aggfunc="mean")
fig = px.imshow(
    pivot_cash,
    aspect="auto",
    title="Heatmap: cash ($m) by company (rows) and month (columns)",
)
fig.show()

# 11) Rolling volatility of oil (3-month rolling std)
market2 = market.copy()
market2["oil_roll_std_3"] = market2["oil_price"].rolling(3).std()
fig = px.line(market2, x="month", y="oil_roll_std_3", title="Oil price volatility (3-month rolling std)")
fig.show()

# 12) Rolling cash change (delta) per company
df_delta = df.sort_values(["company", "month"]).copy()
df_delta["cash_delta"] = df_delta.groupby("company")["cash_musd"].diff()
fig = px.line(df_delta, x="month", y="cash_delta", color="company", title="Monthly cash change ($m)")
fig.show()

# 13) Distribution: monthly cash change
fig = px.histogram(df_delta.dropna(subset=["cash_delta"]), x="cash_delta", color="company", barmode="overlay",
                   title="Distribution of monthly cash changes ($m)")
fig.show()

# 14) Rank over time: who’s leading in cash?
ranks = df.copy()
ranks["rank"] = ranks.groupby("month")["cash_musd"].rank(ascending=False, method="dense")
fig = px.line(ranks, x="month", y="rank", color="company", title="Cash rank over time (1 = richest)")
fig.update_yaxes(autorange="reversed", dtick=1)
fig.show()

print("✅ Loaded rows:", len(df))
print("✅ Companies:", df['company'].nunique())
print("✅ Months:", df['month'].nunique())